# Training Model

## EXP#1

- using only the cleaned dataset
  - imputed the missing value with mode (others with NA below ten is dropped)
- did FE only for encoding and converting to datetime
- classes is VERY imbalanced
- using random forest bcs
  - It handles "Messy" Math
  - It's Robust
  - It tells you "Why"


In [20]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

FE_DATA_PATH = '../data/processed/lending-club-fe.csv'
TARGET_COL = 'loan_status'

In [23]:
df = pd.read_csv(FE_DATA_PATH)
print(df.shape)

(2436, 36)


In [24]:
# Check the Total Balance
print("--- Total Target Balance ---")
print(df['loan_status'].value_counts())
print(df['loan_status'].value_counts(normalize=True))

# Check the size again
print(f"\nTotal Rows: {len(df)}")

--- Total Target Balance ---
loan_status
Fully Paid     2417
Charged Off      19
Name: count, dtype: int64
loan_status
Fully Paid     0.9922
Charged Off    0.0078
Name: proportion, dtype: float64

Total Rows: 2436


In [12]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,issue_d,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App
0,30000,0,22.35,1151.16,3,19,5,100000.0,2,2,...,False,False,False,False,False,False,False,False,True,True
1,40000,1,16.14,975.71,2,13,0,45000.0,1,2,...,False,False,False,False,False,False,False,False,True,True
2,20000,0,7.56,622.68,0,2,10,100000.0,0,2,...,False,False,False,False,False,False,False,False,True,True
3,4500,0,11.31,147.99,1,7,10,38500.0,0,2,...,False,False,False,False,False,False,False,False,True,False
4,8425,0,27.27,345.18,4,24,3,450000.0,1,2,...,False,False,False,False,False,False,False,False,True,True


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2436 entries, 0 to 2435
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   2436 non-null   int64  
 1   term                        2436 non-null   int64  
 2   int_rate                    2436 non-null   float64
 3   installment                 2436 non-null   float64
 4   grade                       2436 non-null   int64  
 5   sub_grade                   2436 non-null   int64  
 6   emp_length                  2436 non-null   int64  
 7   annual_inc                  2436 non-null   float64
 8   verification_status         2436 non-null   int64  
 9   issue_d                     2436 non-null   int64  
 10  loan_status                 2436 non-null   object 
 11  dti                         2436 non-null   float64
 12  open_acc                    2436 non-null   int64  
 13  pub_rec                     2436 

### Split the dataset

In [18]:
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

In [16]:
# Split off the TEST set (10% of total)
# stratify=y ensures the 80/20 ratio is preserved in the test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42, stratify=y
)

# Split the remaining 90% into TRAIN (80%) and VALIDATION (10%)
# The math: 0.10 / 0.90 = 0.1111...
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42, stratify=y_train_val
)

print(f"Train shapes: {X_train.shape}, {y_train.shape}")
print(f"Val shapes:   {X_val.shape}, {y_val.shape}")
print(f"Test shapes:  {X_test.shape}, {y_test.shape}")

Train shapes: (1948, 35), (1948,)
Val shapes:   (244, 35), (244,)
Test shapes:  (244, 35), (244,)


### Training

In [21]:
# Initialize the "Tank"
rf_model = RandomForestClassifier(
    n_estimators=100,      # 100 Trees
    max_depth=10,          # Don't let them grow too deep (prevents overfitting)
    random_state=42,       # Reproducibility
    n_jobs=-1,             # Use all CPU cores (speed!)
    class_weight='balanced' # CRITICAL: Helps the model notice the rare "Bad Loans"
)

In [22]:
# Train the "Tank"
print("Training Random Forest...")
rf_model.fit(X_train, y_train)

# Evaluate on Validation Set
print("Evaluating...")
y_pred = rf_model.predict(X_val)
y_prob = rf_model.predict_proba(X_val)[:, 1] # Probability of Default

# The Scorecard
print(classification_report(y_val, y_pred))
print(f"ROC-AUC Score: {roc_auc_score(y_val, y_prob):.4f}")

Training Random Forest...
Evaluating...
              precision    recall  f1-score   support

 Charged Off       0.00      0.00      0.00         2
  Fully Paid       0.99      1.00      1.00       242

    accuracy                           0.99       244
   macro avg       0.50      0.50      0.50       244
weighted avg       0.98      0.99      0.99       244

ROC-AUC Score: 0.4442


c:\Users\USER\miniconda3\envs\python10\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\USER\miniconda3\envs\python10\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\USER\miniconda3\envs\python10\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", res